## Replace doi references with actual links

In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
import pandas as pd
from collections import Counter

In [4]:
from urllib.parse import urlparse

In [5]:
from functions import log, isInRef

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
headers = { # this is my header
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
}

In [8]:
# dictionary for getting journal name from url
journals_df = pd.read_csv('domain_to_journal.csv', delimiter='\t')
journal_ref = pd.Series(journals_df.journal.values,index=journals_df.domain).to_dict()
# dict: journal_ref['url'] = journal name

In [9]:
df = pd.read_csv('results.2.csv', delimiter= '\t')

In [10]:
df.head()

,link,title,abstract,references
0,https://journals.plos.org/plosone/article?id=1...,Multi-methodological approach for the Quality ...,We set forth to assess the quality of an herba...,"['https://doi.org/10.1016/j.jpba.2011.05.004',..."
1,https://journals.plos.org/plosone/article?id=1...,"Differential COVID-19 testing, admissions, and...",Understanding of COVID-19 acquisition and seve...,"['https://doi.org/10.1111/joim.13117', 'https:..."
2,https://journals.plos.org/plosone/article?id=1...,DAO-CP: Data-Adaptive Online CP decomposition ...,How can we accurately and efficiently decompos...,"['https://doi.org/10.1109/TSP.2017.2690524', '..."
3,https://journals.plos.org/plosone/article?id=1...,Retinal biomarkers of Cerebral Small Vessel Di...,"Cerebral Small Vessel Disease (CSVD), a progre...",['https://doi.org/10.1212/WNL.0000000000007654...
4,https://journals.plos.org/plosone/article?id=1...,Blockchain-based healthcare management system ...,The lack of data outsourcing in healthcare man...,"['https://doi.org/10.3390/sym10100470', 'https..."


In [11]:
# # all references in flat list
# all_references = []
# for sublist in df.references:
#     sub = eval(sublist) # convert from string
#     for item in sub:        
#         all_references.append(item)    

In [12]:
# initialize journal list
journals = []

In [13]:
# collect an example link for every journal
samples = {}
def get_sample_url(url, domain):
    if not(domain in samples.keys()):
        samples[domain] = url

In [15]:
def main():
    # will replace references column
    all_references = []
    counter = 0; # for knowing progress
    for sublist in df.references:
        sub_references = []
        sub = eval(sublist)        
        for link in sub:
            # get request and extract domain
            request = requests.get(link, headers=headers, verify=False)
            domain = urlparse(request.url).netloc
            # append to master list of domains
            journals.append(domain)
            # collect example article for each domain
            get_sample_url(request.url, domain)
            # check if domain is in reference file as some journal
            if isInRef(domain):            
                # append to list of references for this article
                sub_references.append(domain)

                # counter for progress purposes
                counter+=1;
                if counter%25 == 0:
                    print(counter)
            else:
                # do same thing actually lol
                sub_references.append(domain)

                # counter for progress purposes
                counter+=1;
                if counter%25 == 0:
                    print(counter)
                
        # append list of references for this article        
        all_references.append(sub_references)
        
    df.references = all_references
    
    log(f'Ran analyze_ref.3.ipynb succesfully')


In [16]:
main()

25
50
75
100
125
150
175
200
225
250
275
300


In [17]:
df.head()

,link,title,abstract,references
0,https://journals.plos.org/plosone/article?id=1...,Multi-methodological approach for the Quality ...,We set forth to assess the quality of an herba...,"[linkinghub.elsevier.com, linkinghub.elsevier...."
1,https://journals.plos.org/plosone/article?id=1...,"Differential COVID-19 testing, admissions, and...",Understanding of COVID-19 acquisition and seve...,"[onlinelibrary.wiley.com, journals.plos.org, g..."
2,https://journals.plos.org/plosone/article?id=1...,DAO-CP: Data-Adaptive Online CP decomposition ...,How can we accurately and efficiently decompos...,"[ieeexplore.ieee.org, ieeexplore.ieee.org, lin..."
3,https://journals.plos.org/plosone/article?id=1...,Retinal biomarkers of Cerebral Small Vessel Di...,"Cerebral Small Vessel Disease (CSVD), a progre...","[n.neurology.org, journals.sagepub.com, journa..."
4,https://journals.plos.org/plosone/article?id=1...,Blockchain-based healthcare management system ...,The lack of data outsourcing in healthcare man...,"[www.mdpi.com, ieeexplore.ieee.org, linkinghub..."


In [18]:
df.to_csv(r'results.3.csv', sep='\t', index = False)

In [19]:
len(journals)

305

In [20]:
counter = Counter(journals)

In [21]:
counter.most_common()

[('linkinghub.elsevier.com', 72),
 ('onlinelibrary.wiley.com', 36),
 ('link.springer.com', 24),
 ('journals.plos.org', 17),
 ('ieeexplore.ieee.org', 15),
 ('www.nature.com', 13),
 ('www.tandfonline.com', 12),
 ('www.ahajournals.org', 9),
 ('www.mdpi.com', 8),
 ('journals.sagepub.com', 8),
 ('doi.apa.org:443', 8),
 ('www.frontiersin.org', 6),
 ('journals.lww.com', 6),
 ('asa.scitation.org', 6),
 ('www.hindawi.com', 5),
 ('n.neurology.org', 5),
 ('ajph.aphapublications.org', 4),
 ('academic.oup.com', 4),
 ('www.karger.com', 4),
 ('ascelibrary.org', 3),
 ('d.wanfangdata.com.cn', 2),
 ('www.healthaffairs.org', 2),
 ('jech.bmj.com', 2),
 ('jamanetwork.com', 2),
 ('epubs.siam.org', 2),
 ('iovs.arvojournals.org', 2),
 ('pubs.asha.org', 2),
 ('kns.cnki.net', 2),
 ('cmjournal.biomedcentral.com', 1),
 ('bmcecolevol.biomedcentral.com', 1),
 ('gh.bmj.com', 1),
 ('direct.mit.edu', 1),
 ('royalsocietypublishing.org', 1),
 ('svn.bmj.com', 1),
 ('www.ijo.cn', 1),
 ('ojrd.biomedcentral.com', 1),
 ('jnn

In [22]:
samples

{'linkinghub.elsevier.com': 'https://linkinghub.elsevier.com/retrieve/pii/S0731708511002433',
 'link.springer.com': 'https://link.springer.com/article/10.1007/s00216-014-8175-z',
 'onlinelibrary.wiley.com': 'https://onlinelibrary.wiley.com/resolve/doi?DOI=10.1002/jms.4532',
 'www.hindawi.com': 'https://www.hindawi.com/journals/ijac/2021/5209618/',
 'journals.plos.org': 'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0195240',
 'www.frontiersin.org': 'https://www.frontiersin.org/articles/10.3389/fphar.2019.01227/full',
 'www.mdpi.com': 'https://www.mdpi.com/2223-7747/10/1/161',
 'cmjournal.biomedcentral.com': 'https://cmjournal.biomedcentral.com/articles/10.1186/s13020-018-0188-7',
 'www.nature.com': 'https://www.nature.com/articles/s41598-017-17765-5',
 'bmcecolevol.biomedcentral.com': 'https://bmcecolevol.biomedcentral.com/articles/10.1186/1471-2148-10-324',
 'd.wanfangdata.com.cn': 'https://d.wanfangdata.com.cn/periodical/ChlQZXJpb2RpY2FsQ0hJTmV3UzIwMjIwMzIyEg9haG5